In [30]:
import pandas
import numpy
from functools import partial
from sklearn.model_selection import train_test_split

In [31]:
# In iris.data we have a sample of three different types of Iris flowers
data = pandas.read_csv("iris.data", header=None)
data.head()
data[4].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [42]:
# I am going to perform LDA on a training sample of 20% of total observations from types versicolor and virginica
versicolor_samples = data[data[4] == "Iris-versicolor"]
virginica_samples = data[data[4] == "Iris-virginica"]

versicolor_train, versicolor_test = train_test_split(versicolor_samples, test_size=0.8)
virginica_train, virginica_test = train_test_split(virginica_samples, test_size=0.8)

In [ ]:
"""
The classes versicolor and virginica correspond to two distributions D_1 and D_2
Lets assign the prior 1/2 to each distribution in our mixture
    pi_1 = 1/2
    pi_2 = 1/2
    
The distribution of our parametric finite mixture follows as 
    D ~ D_1/2 + D_2/2
    
Then for X ~ D_1/2 + D_2/2, the cdf for X is defined as 
   F_X(x) = P(X_1 <= x_1, X_2 <= x_2, ..., X_p <= x_p)
   
   F_X(x) = P(X_1 <= x_1, X_2 <= x_2, ..., X_p <= x_p | pi_1)P(pi_1) + 
            P(X_1 <= x_1, X_2 <= x_2, ..., X_p <= x_p | pi_2)P(pi_2)
            
   F_X(x) = F_1(x)/2 + F_2(x)/2

The density is just the partial derivative of the cdf with respect to each random variable
   f_X(x) = f_1(x)/2 + f_2(x)/2

If P(pi_1 | X = x) is the conditional probability that an individual would be classified as pi_1 given x
we obtain 
    P(pi_1 | X = x) = P(pi_1)P(X = x | pi_1)/P(X = x)
                    = 1/2 * f_1(x) / f_X(x)
                    = 1/2 * f_1(x) / f_1(x)/2 + f_2(x)/2 # The joint density is a mixture
                    = f_1(x) / f_1(x) + f_2(x)
    P(pi_2 | X = x) = f_2(x) / f_1(x) + f_2(x)
    
A common sense approach to classiyfing an observation would be to check if P(pi_1 | X = x) >= P(pi_2 | X = x).
In English and in the context our problem, this would translate to
    "If the probability of an observation being classified as virginica given its measurements is greater than 
     if it was versicolor, classify it as virginica."

If you divide on both sides and take the log, this rule would be equivalent to 
    log(f_1(x)/f_2(x)) >= 0
"""

In [ ]:
"""
We can derive this classification rule for multivariate Gaussians.

Let pi_1 ~ N(u_1, sigma_1) and pi_2 ~ N(u_2, sigma_2).
LDA introduces homoskedasticity by assuming sigma_1 = sigma_2 = sigma.

The log ratio of two multivariate Gaussians is
    log(f_1(x)/f_2(x)) = -1/2 * (x - u_1).T * inv(sigma) * (x - u_1) + 1/2 * (x - u_2).T * inv(sigma) * (x - u_2)
                       = (u_1 - u_2).T * inv(sigma) * (x - 1/2(u_1 + u_2)) # This is known as Fisher's Rule
                       
Unlike the Bayes Discriminant Approach, Fisher derived this classification rule by creating a line such that
if all points were projected on it maximially seperated according to their scaled squared mean difference.
"""

In [45]:
versicolor_train_X = versicolor_train.iloc[:, 0:4].to_numpy()
virginica_train_X = virginica_train.iloc[:, 0:4].to_numpy()
versicolor_test_X = versicolor_test.iloc[:, 0:4].to_numpy()
virginica_test_X = virginica_test.iloc[:, 0:4].to_numpy()

In [46]:
versicolor_train_S = numpy.cov(versicolor_train_X.T)
virginica_train_S = numpy.cov(virginica_train_X.T)
S_pooled = (versicolor_train_S.shape[0] - 1) / (versicolor_train_S.shape[0] + virginica_train_S.shape[0] - 2) * versicolor_train_S + (virginica_train_S.shape[0] - 1) / (versicolor_train_S.shape[0] + virginica_train_S.shape[0] - 2) * virginica_train_S
S_pooled_inv = numpy.linalg.inv(S_pooled)
xbar_virginica = virginica_train_X.mean(axis=0)
xbar_versicolor = versicolor_train_X.mean(axis=0)

In [47]:
def fishersDiscriminantRule(S_pooled_inv: numpy.ndarray, xbar_1: numpy.ndarray, xbar_2: numpy.ndarray, x: numpy.ndarray):
    return (xbar_1 - xbar_2).T @ S_pooled_inv @ (x - 1/2 * (xbar_1 + xbar_2)).reshape(-1,1) >= 0

In [48]:
apply_function = partial(fishersDiscriminantRule, S_pooled_inv, xbar_virginica, xbar_versicolor)

In [49]:
versicolor_classifications = numpy.apply_along_axis(apply_function, 1, versicolor_test_X)
virginica_classifications = numpy.apply_along_axis(apply_function, 1, virginica_test_X)

In [51]:
print("Versicolor classification rate: ", sum(versicolor_classifications == False) / versicolor_test_X.shape[0])
print("Virginica classification rate: ", sum(virginica_classifications == True) / virginica_test_X.shape[0])

Versicolor classification rate:  [0.95]
Virginica classification rate:  [0.925]
